# Assignment 3: Shortest Path Algorithms

This notebook implements and compares four fundamental shortest path algorithms:
- **Dijkstra's Algorithm** - Single-source shortest path for graphs with non-negative weights
- **Bellman-Ford Algorithm** - Single-source shortest path that handles negative weights
- **Floyd-Warshall Algorithm** - All-pairs shortest path using dynamic programming
- **Johnson's Algorithm** - All-pairs shortest path optimized using reweighting

**Course:** Design and Analysis of Algorithms

**Semester:** 5

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## 1. Required Libraries

Import necessary Python libraries:
- `numpy` - For numerical operations
- `pandas` - For displaying distance matrices in tabular format
- `matplotlib.pyplot` - For potential graph visualizations

In [2]:
graph={}
vertices,edges=0,0
print("""Enter vertices, Edges and egde weight in the folloing format:
Example Input Format:
8     -> Number of lines to follow (edges + 1)
5 7   -> Number of vertices and edges
1 2 2 -> Edge from vertex 1 to vertex 2 with weight 2
1 3 4
2 3 1
2 4 7
3 5 3
4 5 1
5 4 2

You are supposed to enter edges one by one in the same format as above.
""")


lines = iter(input().strip() for _ in range(int(input("Enter number of lines to follow (edges + 1): ")) ))

first_line = next(lines)

vertices, edges = map(int, first_line.split())

for _ in range(edges):
    u, v, w = map(int, next(lines).split())
    if u not in graph:
        graph[u] = []
    graph[u].append((v, w))

print("no of vertices:", vertices)
print("no of edges:", edges)
print("Graph representation (Adjacency List):")

for vertex in graph:
    print("vertex", vertex, "->", graph[vertex])
    


Enter vertices, Edges and egde weight in the folloing format:
Example Input Format:
8     -> Number of lines to follow (edges + 1)
5 7   -> Number of vertices and edges
1 2 2 -> Edge from vertex 1 to vertex 2 with weight 2
1 3 4
2 3 1
2 4 7
3 5 3
4 5 1
5 4 2

You are supposed to enter edges one by one in the same format as above.

no of vertices: 5
no of edges: 7
Graph representation (Adjacency List):
vertex 1 -> [(2, 2), (3, 4)]
vertex 2 -> [(3, 1), (4, 7)]
vertex 3 -> [(5, 3)]
vertex 4 -> [(5, 1)]
vertex 5 -> [(4, 2)]


## 2. Graph Input (Interactive)

This cell allows you to input a custom graph interactively. The graph is represented as an adjacency list where each vertex maps to a list of (neighbor, weight) tuples.

**Input Format:**
```
Number of lines to follow (edges + 1)
vertices edges
u v w  (edge from u to v with weight w)
...
```

**Example:**
```
8
5 7
1 2 2
1 3 4
2 3 1
2 4 7
3 5 3
4 5 1
5 4 2
```

In [3]:
graph={
    1: [(2, 2), (3, 4)],
    2: [(3, 1), (4, 7)],
    3: [(5, 3)],
    4: [(5, 1)],
    5: [(4, 2)]
}
vertices=5
edges=7

## 3. Hardcoded Example Graph

A predefined graph for quick testing. This graph has:
- **5 vertices** (numbered 1 to 5)
- **7 edges** (some directed edges with positive weights)

Graph structure:
- Vertex 1 → [(2, weight=2), (3, weight=4)]
- Vertex 2 → [(3, weight=1), (4, weight=7)]
- Vertex 3 → [(5, weight=3)]
- Vertex 4 → [(5, weight=1)]
- Vertex 5 → [(4, weight=2)]

Note: This graph contains a cycle (4→5→4) but no negative cycles.

## 4. Dijkstra's Algorithm

**Purpose:** Find the shortest path from a single source vertex to all other vertices in a graph with **non-negative edge weights**.

**Algorithm Overview:**
1. Initialize all distances to infinity, except the source (distance = 0)
2. Use a min-heap (priority queue) to process vertices in order of their distance
3. For each vertex, relax all outgoing edges
4. Continue until all vertices are processed

**Time Complexity:** O((V + E) log V) with a proper heap implementation

**Limitations:** Cannot handle negative edge weights

This implementation uses a simple list-based heap with sorting for demonstration purposes.

In [20]:
def dijkstra(graph, start):
    heap =[] 
    heap.append((0, start))  # (distance, vertex)
    distances = {}
    for vertex in range(1, vertices + 1):
        distances[vertex] = float('infinity')
    distances[start] = 0

    while heap:
        current_distance, current_vertex = heap.pop(0)

        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph.get(current_vertex, []):
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heap.append((distance, neighbor))
                heap.sort()  # Maintain the heap property (Min-Heap)
    return distances

print("\n RUNNING DIJKSTRA'S ALGORITHM ...")
start_vertex = 1
distances = dijkstra(graph, start_vertex)
print(f"Shortest distances from vertex {start_vertex}:")
for vertex in range(1, vertices + 1):
    print(f"Vertex {vertex}: {distances[vertex]}")



 RUNNING DIJKSTRA'S ALGORITHM ...
Shortest distances from vertex 1:
Vertex 1: 0
Vertex 2: 2
Vertex 3: 3
Vertex 4: 8
Vertex 5: 6


## 5. Bellman-Ford Algorithm

**Purpose:** Find the shortest path from a single source vertex to all other vertices, even with **negative edge weights**.

**Algorithm Overview:**
1. Initialize all distances to infinity, except the source (distance = 0)
2. Relax all edges repeatedly (V-1 times)
3. Check for negative weight cycles by attempting one more relaxation
4. If any distance can still be reduced, a negative cycle exists

**Time Complexity:** O(V × E)

**Advantages over Dijkstra:**
- Can handle negative edge weights
- Can detect negative weight cycles

**Use Case:** Graphs with negative weights but no negative cycles (e.g., financial arbitrage detection)

In [21]:
def bellman_ford(graph, source):
    distances=[]
    for vertex in range(1, vertices + 1):
        distances.append(float('infinity'))
    distances[source] = 0  

    # Relax edges repeatedly
    for _ in range(vertices - 1):
        for u in graph:           # u=current vertex
            for v, w in graph[u]: # v=neighbor, w=weight

                # Relaxation step
                if distances[u - 1] + w < distances[v - 1]:
                    distances[v - 1] = distances[u - 1] + w


    # Check for negative-weight cycles
    for u in graph: 
        for v, w in graph[u]:
            if distances[u - 1] + w < distances[v - 1]:
                print("Graph contains negative weight cycle")
                return None

    return distances


print("\n RUNNING BELLMAN-FORD ALGORITHM ...")
start_vertex = 1
distances = bellman_ford(graph, start_vertex - 1)
if distances:
    print("Shortest distances from vertex", start_vertex, "using Bellman-Ford:")
    for vertex in range(1, vertices + 1):
        print("Vertex", vertex, ":", distances[vertex - 1])


 RUNNING BELLMAN-FORD ALGORITHM ...
Shortest distances from vertex 1 using Bellman-Ford:
Vertex 1 : 0
Vertex 2 : 2
Vertex 3 : 3
Vertex 4 : 8
Vertex 5 : 6


## 6. Floyd-Warshall Algorithm

**Purpose:** Find the shortest paths between **all pairs of vertices** using dynamic programming.

**Algorithm Overview:**
1. Initialize a distance matrix: dist[i][j] = weight of edge (i,j), or ∞ if no edge exists
2. For each intermediate vertex k, update distances:
   - dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
3. After considering all vertices as intermediates, the matrix contains all shortest paths

**Time Complexity:** O(V³)

**Space Complexity:** O(V²)

**Advantages:**
- Simple implementation
- Computes all-pairs shortest paths in one run
- Works with negative edges (but not negative cycles)

**Best suited for:** Dense graphs where you need distances between all vertex pairs

The output is displayed as a pandas DataFrame for better readability.

In [22]:
def floydwarshell(graph):
    # step 1: Initialize distance matrix
    dist =[]
    for i in range(vertices):
        dist.append([])
        for j in range(vertices):
            if i == j:
                dist[i].append(0)
            else:
                dist[i].append(float('infinity'))

    #step 2: Initialize distances based on graph edges            
    for u in graph:
        for v, w in graph[u]:
            dist[u - 1][v - 1] = w

    # step 3: Update distance matrix
    for i in range(vertices):          # intermediate 
        for j in range(vertices):      # start       
            for k in range(vertices):  # end         

                # Relaxation step 
                if dist[j][i] + dist[i][k] < dist[j][k]:
                    dist[j][k] = dist[j][i] + dist[i][k]
    return dist

distances = floydwarshell(graph)

def print_matrix_pandas(dist):
    # Convert INF to a readable "INF"
    df = pd.DataFrame(dist)
    df.replace([float('inf'), np.inf], "INF", inplace=True)

    # Rename rows and columns (1-based)
    df.index = [f"{i}" for i in range(1, len(dist) + 1)]
    df.columns = [f"{j}" for j in range(1, len(dist) + 1)]

    print(df)

print("\n RUNNING FLOYD-WARSHALL ALGORITHM ...")
print_matrix_pandas(distances)



 RUNNING FLOYD-WARSHALL ALGORITHM ...
     1    2    3  4  5
1  0.0  2.0  3.0  8  6
2  INF  0.0  1.0  6  4
3  INF  INF  0.0  5  3
4  INF  INF  INF  0  1
5  INF  INF  INF  2  0


### Graph Visualization

Quick display of the current graph structure in adjacency list format.

In [8]:
print(graph)

{1: [(2, 2), (3, 4)], 2: [(3, 1), (4, 7)], 3: [(5, 3)], 4: [(5, 1)], 5: [(4, 2)]}


## 7. Johnson's Algorithm

**Purpose:** Efficiently compute **all-pairs shortest paths** in sparse graphs, especially those with **negative edge weights**.

**Algorithm Overview:**
1. Add a new vertex (source s) connected to all vertices with weight 0
2. Run Bellman-Ford from s to compute function h(v) for reweighting
3. Reweight all edges: w'(u,v) = w(u,v) + h(u) - h(v)
4. Remove the added vertex
5. Run Dijkstra from each vertex on the reweighted graph
6. Convert distances back using: d(u,v) = d'(u,v) - h(u) + h(v)

**Time Complexity:** O(V² log V + VE)

**Why it's efficient:**
- Reweighting makes all edges non-negative (allowing Dijkstra)
- For sparse graphs (E << V²), this is faster than Floyd-Warshall O(V³)

**Best suited for:** Sparse graphs with negative weights but no negative cycles

This section includes helper functions `bellman_fordH` and `dijkstraH` specifically adapted for Johnson's algorithm.

In [23]:
def bellman_fordH(graph, source, vertices):
    distances = {}
    # Initialize all vertices that appear in the graph
    for u in graph:
        distances[u] = float('infinity')
        for v, w in graph[u]:
            if v not in distances:
                distances[v] = float('infinity')
    
    distances[source] = 0  

    # Relax edges repeatedly
    for _ in range(vertices - 1):
        for u in graph:
            if distances[u] != float('infinity'):
                for v, w in graph[u]:
                    # Relaxation step
                    if distances[u] + w < distances[v]:
                        distances[v] = distances[u] + w

    # Check for negative-weight cycles
    for u in graph: 
        if distances[u] != float('infinity'):
            for v, w in graph[u]:
                if distances[u] + w < distances[v]:
                    print("Graph contains negative weight cycle")
                    return None

    return distances

def dijkstraH(graph, start,vertices):
    heap =[] 
    heap.append((0, start))  # (distance, vertex)
    distances = {}
    for vertex in range(1, vertices + 1):
        distances[vertex] = float('infinity')
    distances[start] = 0

    while heap:
        current_distance, current_vertex = heap.pop(0)

        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph.get(current_vertex, []):
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heap.append((distance, neighbor))
                heap.sort()  # Maintain the heap property (Min-Heap)
    return distances


In [24]:
def johnson(graph):
    #step 1: add new vertex '0' 
    newGraph={}
    newGraph[0] = []

    #step 2:  connected to all other vertices with edge weight 0
    for vertex in range(1, vertices + 1):
        newGraph[0].append((vertex, 0))
        newGraph[vertex] = graph.get(vertex, [])
    # step 3: run bellman ford from new vertex '0'
    bellmanDist= bellman_fordH(newGraph, 0,vertices)
    if bellmanDist is None:
        print("Graph contains negative weight cycle. Johnson's algorithm cannot proceed.")
        return None
    # step 4: reweight edges
    reweightedGraph={}
    for u in graph:
        reweightedGraph[u] = []
        for v, w in graph[u]:
            newWeight = w + bellmanDist[u] - bellmanDist[v]
            reweightedGraph[u].append((v, newWeight))
    # step 5: remove the added vertex '0' (not needed anymore as we have reweighted the graph) 
    # step 6: run dijkstra for each vertex
    ActualDIst={}
    for v in range(1, vertices + 1):
        dijkstraDist = dijkstraH(reweightedGraph, v,vertices)
        ActualDIst[v] = {}
        for u in dijkstraDist:
            ActualDIst[v][u] = dijkstraDist[u] - bellmanDist[v] + bellmanDist[u]
    return ActualDIst


def print_matrix_pandas(dist):
    # Convert INF to a readable "INF"
    df = pd.DataFrame(dist)
    df.replace([float('inf'), np.inf], "INF", inplace=True)

    # Rename rows and columns (1-based)
    df.index = [f"{i}" for i in range(1, len(dist) + 1)]
    df.columns = [f"{j}" for j in range(1, len(dist) + 1)]
    print("\nDistance Matrix:")
    print(df)


all_pairs_distances = johnson(graph)
if all_pairs_distances:
    print("\n RUNNING JOHNSON'S ALGORITHM ...")
    for u in all_pairs_distances:
        for v in all_pairs_distances[u]:
            print(f"Distance from vertex {u} to vertex {v}: {all_pairs_distances[u][v]}")

    # Convert to matrix form for better visualization
    distance_matrix = []
    for i in range(1, vertices + 1):
        row = []
        for j in range(1, vertices + 1):
            row.append(all_pairs_distances[i][j])
        distance_matrix.append(row)
    print_matrix_pandas(distance_matrix)
    


 RUNNING JOHNSON'S ALGORITHM ...
Distance from vertex 1 to vertex 1: 0
Distance from vertex 1 to vertex 2: 2
Distance from vertex 1 to vertex 3: 3
Distance from vertex 1 to vertex 4: 8
Distance from vertex 1 to vertex 5: 6
Distance from vertex 2 to vertex 1: inf
Distance from vertex 2 to vertex 2: 0
Distance from vertex 2 to vertex 3: 1
Distance from vertex 2 to vertex 4: 6
Distance from vertex 2 to vertex 5: 4
Distance from vertex 3 to vertex 1: inf
Distance from vertex 3 to vertex 2: inf
Distance from vertex 3 to vertex 3: 0
Distance from vertex 3 to vertex 4: 5
Distance from vertex 3 to vertex 5: 3
Distance from vertex 4 to vertex 1: inf
Distance from vertex 4 to vertex 2: inf
Distance from vertex 4 to vertex 3: inf
Distance from vertex 4 to vertex 4: 0
Distance from vertex 4 to vertex 5: 1
Distance from vertex 5 to vertex 1: inf
Distance from vertex 5 to vertex 2: inf
Distance from vertex 5 to vertex 3: inf
Distance from vertex 5 to vertex 4: 2
Distance from vertex 5 to vertex 5: 

## 8. Testing and Performance Comparison

This section defines multiple test graphs to compare the algorithms' behavior on different graph types:

1. **Sparse Graph** (E ≈ V): Few edges, chain-like structure
2. **Dense Graph** (E ≈ V²): Many edges, highly connected
3. **Mixed Negative Graph**: Contains negative edge weights to test Bellman-Ford and Johnson's algorithms

Each algorithm is run on all three graphs to demonstrate:
- Correctness across different graph structures
- How algorithms handle various edge weight distributions
- Performance characteristics (though timing is not measured here)

In [25]:
# Graph 1: Sparse Graph (10 vertices, ~10 edges, E ≈ V)
graph1 = {
    1: [(2, 5), (3, 3)],
    2: [(4, 6)],
    3: [(5, 2)],
    4: [(6, 4)],
    5: [(7, 8)],
    6: [(8, 3)],
    7: [(9, 5)],
    8: [(10, 7)],
    9: [(10, 2)],
    10: []
}
vertices1 = 10
edges1 = 10

# Graph 2: Dense Graph (10 vertices, ~45 edges, E ≈ V²)
# For demonstration with smaller vertices (full dense would be 100-200 vertices)
graph2 = {
    1: [(2, 3), (3, 8), (4, 5), (5, 2), (6, 9), (7, 4), (8, 6), (9, 7)],
    2: [(3, 2), (4, 4), (5, 6), (6, 3), (7, 5), (8, 8), (9, 2), (10, 4)],
    3: [(4, 1), (5, 7), (6, 2), (7, 9), (8, 3), (9, 6), (10, 5)],
    4: [(5, 4), (6, 6), (7, 2), (8, 7), (9, 3), (10, 8)],
    5: [(6, 5), (7, 3), (8, 4), (9, 9), (10, 2)],
    6: [(7, 6), (8, 2), (9, 5), (10, 7)],
    7: [(8, 8), (9, 4), (10, 3)],
    8: [(9, 6), (10, 5)],
    9: [(10, 2)],
    10: []
}
vertices2 = 10
edges2 = 44

# Graph 3: Mixed Graph with negative weights (15 vertices, random weights including negatives)
# Suitable for Bellman-Ford & Johnson's algorithms
graph3 = {
    1: [(2, 4), (3, -2)],
    2: [(3, 3), (4, 2), (5, -1)],
    3: [(5, 5), (6, 4)],
    4: [(6, -3), (7, 6)],
    5: [(7, 2), (8, -4)],
    6: [(8, 3), (9, 5)],
    7: [(9, -2), (10, 4)],
    8: [(10, 6), (11, -1)],
    9: [(11, 3), (12, 2)],
    10: [(12, -3), (13, 5)],
    11: [(13, 4), (14, -2)],
    12: [(14, 3), (15, 6)],
    13: [(15, -1)],
    14: [(15, 2)],
    15: []
}
vertices3 = 15
edges3 = 28

# To test with any graph, uncomment the desired graph:
# graph = graph1
# vertices = vertices1
# edges = edges1

# graph = graph2
# vertices = vertices2
# edges = edges2

# graph = graph3
# vertices = vertices3
# edges = edges3



### 8.1 Test Graph Definitions

Three different graph types are defined:

- **graph1 (Sparse):** 10 vertices, ~10 edges - resembles a chain or tree structure
- **graph2 (Dense):** 10 vertices, ~44 edges - highly interconnected
- **graph3 (Mixed Negative):** 15 vertices, 28 edges with some negative weights

Uncomment the desired graph assignment at the bottom to test algorithms individually with a specific graph.

In [ ]:
graphs = [
    {"name": "Sparse Graph", "graph": graph1, "vertices": vertices1, "edges": edges1},
    {"name": "Dense Graph", "graph": graph2, "vertices": vertices2, "edges": edges2},
    {"name": "Mixed Negative Graph", "graph": graph3, "vertices": vertices3, "edges": edges3}
]

### 8.2 Test Graph Collection

Package all test graphs into a list with metadata for batch testing.

In [ ]:
def run_all_algorithms(graph_data):
    name = graph_data["name"]
    graph = graph_data["graph"]
    vertices = graph_data["vertices"]
    edges = graph_data["edges"]

    print(f"\n--- Testing {name} ---")

    start_vertex = 1

    # ---------------- Dijkstra ----------------
    print("\n RUNNING DIJKSTRA'S ALGORITHM ...")
    distances = dijkstra(graph, start_vertex)
    for v in range(1, vertices + 1):
        print(f"Vertex {v}: {distances[v]}")

    # ---------------- Bellman-Ford ----------------
    print("\n RUNNING BELLMAN-FORD ALGORITHM ...")
    distances = bellman_ford(graph, start_vertex - 1)
    if distances:
        for v in range(1, vertices + 1):
            print(f"Vertex {v}: {distances[v - 1]}")

    # ---------------- Floyd-Warshall ----------------
    print("\n RUNNING FLOYD-WARSHALL ALGORITHM ...")
    fw_matrix = floydwarshell(graph)
    print_matrix_pandas(fw_matrix)

    # ---------------- Johnson ----------------
    print("\n RUNNING JOHNSON'S ALGORITHM ...")
    all_pairs = johnson(graph)
    if all_pairs:
        for u in all_pairs:
            for v in all_pairs[u]:
                print(f"Distance {u} → {v}: {all_pairs[u][v]}")

        # Convert to matrix
        matrix = [[all_pairs[i][j] for j in range(1, vertices + 1)]
                   for i in range(1, vertices + 1)]
        print_matrix_pandas(matrix)

for graph_data in graphs:
    run_all_algorithms(graph_data)
    


--- Testing Sparse Graph ---

 RUNNING FLOYD-WARSHALL ALGORITHM ...


IndexError: list assignment index out of range

### 8.3 Comprehensive Test Runner

This cell runs all four algorithms on each test graph and displays the results:

**For each graph, it executes:**
1. **Dijkstra** - Single-source shortest paths from vertex 1
2. **Bellman-Ford** - Single-source shortest paths from vertex 1 (handles negatives)
3. **Floyd-Warshall** - All-pairs shortest paths (displayed as matrix)
4. **Johnson** - All-pairs shortest paths (displayed as both list and matrix)

**Expected Behavior:**
- On graphs with negative weights, Dijkstra may produce incorrect results
- Bellman-Ford and Johnson's should handle negative weights correctly
- All algorithms should agree on results for non-negative graphs

Run this cell to see a comprehensive comparison across all test cases.